In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from visual_behavior import database as db

pd.set_option('display.max_columns', 500)

import visual_behavior.visualization.utils as utils
import visual_behavior.data_access.loading as loading
import visual_behavior.data_access.filtering as filtering
import visual_behavior.ophys.response_analysis.utilities as ut
from visual_behavior.ophys.response_analysis.response_analysis import ResponseAnalysis

## define the acceptable failure tags

In [2]:
acceptable_fail_tags = [
    'd_prime_peak', # this means the animal didn't perform the task well - was not discriminating
    'num_contingent_trials', # this also means the animal didn't perform the task well - likely too many aborted trials
    'validate_omitted_flashes_are_omitted', # this was due to a recurrent camstim bug that turned out having little effect on data integrity
]

## load the experiment_table

In [3]:
experiment_table = loading.get_filtered_ophys_experiment_table(include_failed_data=True)

In [4]:
experiment_table['experiment_workflow_state'].unique()

array(['failed', 'passed'], dtype=object)

In [5]:
experiment_table.query('experiment_workflow_state == "failed"').sample(5)

,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at,super_container_id,cre_line,session_tags,failure_tags,session_type_exposure_number,prior_exposures_to_image_set,prior_exposures_to_omissions,model_outputs_available,has_cell_matching,location,has_events,session_number
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
875808255,875259383,875471358,1018027605,VisualBehaviorMultiscope,container_qc,failed,20190524_449653_Ophys6,OPHYS_6_images_A,MESO.1,2019-05-24 12:37:09.824183,846102910,837628436,M,126.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],79,VISl,NaN,837628436,Vip-IRES-Cre,NaN,"d_prime_peak,num_contingent_trials",0,17.0,6.0,False,False,Vip_VISl_79,True,6.0
1049853339,1049740675,1049767272,1046576791,VisualBehaviorMultiscope,holding,failed,20200911_523922_ophys2retake,OPHYS_2_images_A_passive,MESO.1,2020-09-11 11:47:43.237104,1025266430,1022744256,M,179.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],227,VISl,NaN,1022744256,Vip-IRES-Cre,NaN,"percent_change_intensity,percent_change_intens...",1,49.0,5.0,False,True,Vip_VISl_227,True,2.0
929603823,927620400,927764490,1018028328,VisualBehaviorMultiscope,failed,failed,20190820_Ophys4_retake_4,OPHYS_4_images_B,MESO.1,2019-08-20 12:39:37.863363,847875381,840542948,M,193.0,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,['Ai93(TITL-GCaMP6f)'],"['Slc17a7-IRES2-Cre', 'Camk2a-tTA']",171,VISl,NaN,840542948,Slc17a7-IRES2-Cre,NaN,NaN,2,5.0,8.0,False,False,Slc17a7_VISl_171,False,4.0
845777913,845235947,845404780,1018027567,VisualBehaviorMultiscope,failed,failed,20190403_429956_Ophys5Passive,OPHYS_4_images_B,MESO.1,2019-04-03 09:01:20.000000,802369734,791855403,F,173.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Vip-IRES-Cre'],75,VISp,NaN,791855403,Vip-IRES-Cre,mesoscope_pilot_datacube,NaN,1,1.0,5.0,False,True,Vip_VISp_75,True,4.0
988542657,987896411,988099332,975371413,VisualBehaviorTask1B,container_qc,failed,20191120_477202_6imagesA,OPHYS_6_images_A,CAM2P.3,2019-11-20 21:00:49.000000,918315422,910248639,M,171.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,['Ai148(TIT2L-GC6f-ICL-tTA2)'],['Sst-IRES-Cre'],275,VISp,NaN,910248639,Sst-IRES-Cre,NaN,d_prime_peak,1,3.0,7.0,False,False,Sst_VISp_275,False,6.0


## define a function to query mouseseeks to get all experiments that have a given failure tag

In [6]:
def get_experiments_with_fail_tag(fail_tag):
    '''
    query mouseseeks to get all experiments that have a given failure tag
    note that this will return all experiments with that fail tag, even if it also has other fail tags
    '''
    mouseseeks = db.Database('mouseseeks')
    documents = list(mouseseeks['db']['ophys_session_log'].find({'failure_tags':{"$in":[fail_tag]}}))
    mouseseeks.close()
    return pd.DataFrame(documents)

## get all experiments that have any of the acceptable failure tags
drop duplicates to avoid getting an experiment more than once if it has more than one acceptable 

In [7]:
documents = pd.DataFrame()
# iterate over every acceptable failure tag
for fail_tag in acceptable_fail_tags:
    # get all experiments with this tag
    df = get_experiments_with_fail_tag(fail_tag)
    
    # concatenate with existing list of experiments, then drop duplicates
    documents = pd.concat([documents, df]).drop_duplicates(subset = ['id','specimen_id']).reset_index(drop=True)
documents.sample(5)

,_id,id,isi_experiment,name,workflow_state,specimen_id,date_of_acquisition,depth,project_code,area,storage_directory,ophys_parent_session,operator,rig,ophys_experiments,external_specimen_name,genotype,qc_status,failure_tags,flags,overrides,qc_reports,notes,session_tags
51,5d5ae73017ec6f430f4419a2,926488384,{'storage_directory': '/allen/programs/braintv...,20190819_453909_Ophys6_retake,uploaded,840542948,2019-08-19 11:05:58.134,NaN,VisualBehaviorMultiscope,None,/allen/programs/braintv/production/visualbehav...,{'storage_directory': '/allen/programs/braintv...,sara.kivikas,MESO.1,[{'storage_directory': '/allen/programs/braint...,453909,Slc17a7-IRES2-Cre;Camk2a-tTA;Ai93,"{'status': 'success', 'message': ''}","[parent_averaged_depth_image_2, parent_average...","[{'lims_id': '926488384', 'notes': 'FLAG: Hig...","[{'lims_id': '926488384', 'notes': 'OVERRIDE: ...",[{'submit_datetime': 2019-08-22 09:17:22.26000...,NaN,NaN
189,5d6fd50917ec6f430f442479,938094182,{'storage_directory': '/allen/programs/braintv...,20190904_459773_2imagesApassive,uploaded,854612265,2019-09-04 15:01:19.000,175.0,VisualBehavior,VISp,/allen/programs/braintv/production/visualbehav...,{'storage_directory': '/allen/programs/braintv...,xana.waughman,CAM2P.5,[{'storage_directory': '/allen/programs/braint...,459773,Slc17a7-IRES2-Cre;Camk2a-tTA;Ai93,"{'status': 'success', 'message': ''}",[validate_omitted_flashes_are_omitted],[],"[{'lims_id': '938094182', 'notes': 'OVERRIDE: ...",[{'submit_datetime': 2019-09-12 13:34:16.68100...,NaN,NaN
16,5d011e1509ed44852535cdfb,885229423,{'storage_directory': '/allen/programs/braintv...,20190612_442147_1imagesA,uploaded,830896328,2019-06-12 15:30:43.000,75.0,VisualBehavior,VISp,/allen/programs/braintv/production/visualbehav...,{'storage_directory': '/allen/programs/braintv...,shiellac,CAM2P.4,[{'storage_directory': '/allen/programs/braint...,442147,Chat-IRES-Cre-neo;Chrm2-tdT;Ai162,"{'status': 'success', 'message': ''}",[d_prime_peak],[],[],[{'submit_datetime': 2019-06-13 08:13:17.58200...,NaN,[visbehneuromodax]
20,5cfe9ab909ed44852535cb79,883619540,{'storage_directory': '/allen/programs/braintv...,20190610_451787_4imagesB,uploaded,837581585,2019-06-10 10:58:39.000,NaN,VisualBehaviorMultiscope,None,/allen/programs/braintv/production/visualbehav...,{'storage_directory': '/allen/programs/braintv...,india.kato,MESO.1,[{'storage_directory': '/allen/programs/braint...,451787,Slc17a7-IRES2-Cre;Camk2a-tTA;Ai93,"{'status': 'success', 'message': ''}","[percent_change_intensity, percent_change_inte...",[],"[{'lims_id': '884271249', 'notes': 'OVERRIDE: ...",[{'submit_datetime': 2019-06-11 12:12:15.88600...,Animal did not behave\r\nintensity drift,NaN
89,5dbcaaf217ec6f430f444c91,975460534,{'storage_directory': '/allen/programs/braintv...,20191101_479835_3B,uploaded,920850703,2019-11-01 21:51:03.000,175.0,VisualBehaviorTask1B,VISp,/allen/programs/braintv/production/visualbehav...,{'storage_directory': '/allen/programs/braintv...,joshl,CAM2P.5,[{'storage_directory': '/allen/programs/braint...,479835,Slc17a7-IRES2-Cre;Camk2a-tTA;Ai93,"{'status': 'success', 'message': ''}","[d_prime_peak, num_contingent_trials]","[{'lims_id': '975542757', 'notes': 'FLAG: int...",[],[{'submit_datetime': 2019-11-04 09:24:02.14200...,NaN,NaN


## define a function to check to see if 'failure_tags' contains only acceptable tags

In [10]:
def only_acceptable_fail_tags_in_record(document, acceptable_fail_tags):
    # checks to see if the experiment had any failure tags beyond those listed as 'acceptable'
    # returns True if all fail tags are in set of acceptible fail tags, False otherwise
    return len(set(document['failure_tags']).union(set(acceptable_fail_tags)) - set(acceptable_fail_tags)) == 0

## apply the function, assign boolean output to a column called "only_acceptable_fail_tags"

In [11]:
documents['only_acceptable_fail_tags'] = documents.apply(
    only_acceptable_fail_tags_in_record, 
    axis=1, 
    acceptable_fail_tags=acceptable_fail_tags
)

## Mesoscope sessions
A mesoscope session consists of 8 experiments  
In some cases, only acceptable fail tags apply to all experiments. This case is straightforward and allows all to be unfailed  
In other cases, at least one experiment had an unacceptable fail tag. But mouseseeks does not, as far as I can tell, track which fail tags were associated with which experiment. For now, we are going to unfail all sessions that have at least one unacceptable fail tag. The individual failed experiments can be removed later.

In [12]:
len(documents.query('rig == "MESO.1" and only_acceptable_fail_tags == True'))

7

In [13]:
len(documents.query('rig == "MESO.1" and only_acceptable_fail_tags == False'))

19

## Final conditions for unfailing:
* Session has only acceptable fail tags  
*or*
* Session is a mesoscope session with at least one acceptable fail tag (individual failing experiments to be removed later)

In [14]:
def unfail(document):
    return document['only_acceptable_fail_tags'] or document['rig'] == "MESO.1"
documents['unfail'] = documents.apply(unfail, axis=1)
documents['unfail'].value_counts()

True     151
False     42
Name: unfail, dtype: int64

## Save the sessions to unfail to a CSV file

In [15]:
sessions_to_unfail = documents.query('unfail').rename(columns={'id': 'ophys_session_id'})

columns_to_save = [
    'ophys_session_id',
    'specimen_id',
    'date_of_acquisition',
    'depth',
    'project_code',
    'area',
    'rig',
    'failure_tags',
    'only_acceptable_fail_tags',
    'unfail',
]
sessions_to_unfail[columns_to_save].to_csv('/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/2021.01.20_sessions_to_unfail.csv', index=False)

In [16]:
sessions_to_unfail[columns_to_save]

,ophys_session_id,specimen_id,date_of_acquisition,depth,project_code,area,rig,failure_tags,only_acceptable_fail_tags,unfail
0,856295914,820871408,2019-04-24 16:49:34,375.0,VisualBehavior,VISp,CAM2P.3,[d_prime_peak],True,True
1,857232455,820871408,2019-04-25 17:20:49,375.0,VisualBehavior,VISp,CAM2P.3,[d_prime_peak],True,True
3,857202651,823826986,2019-04-25 17:00:04,175.0,VisualBehavior,VISp,CAM2P.4,[d_prime_peak],True,True
6,863527124,823826986,2019-05-06 20:07:55,175.0,VisualBehavior,VISp,CAM2P.4,[d_prime_peak],True,True
7,865275609,821470815,2019-05-09 18:12:00,175.0,VisualBehavior,VISp,CAM2P.4,[d_prime_peak],True,True
...,...,...,...,...,...,...,...,...,...,...
185,851426367,823826986,2019-04-15 15:31:58,175.0,VisualBehavior,VISp,CAM2P.4,[validate_omitted_flashes_are_omitted],True,True
189,938094182,854612265,2019-09-04 15:01:19,175.0,VisualBehavior,VISp,CAM2P.5,[validate_omitted_flashes_are_omitted],True,True
190,973776534,920850703,2019-10-29 21:27:12,175.0,VisualBehaviorTask1B,VISp,CAM2P.5,[validate_omitted_flashes_are_omitted],True,True
191,1010897308,991762388,2020-02-28 16:38:10,175.0,VisualBehaviorTask1B,VISp,CAM2P.4,[validate_omitted_flashes_are_omitted],True,True


In [17]:
mouseseeks = db.Database('mouseseeks')
documents = list(mouseseeks['db']['ophys_session_log'].find({'id':842023261}))
mouseseeks.close()

In [18]:
documents[0]['failure_tags']

['z_drift_corr_um_diff']

In [19]:
documents[0]

{'_id': ObjectId('5c9b9a9309ed4485253592f5'),
 'id': 842023261,
 'isi_experiment': {'storage_directory': '/allen/programs/braintv/production/neuralcoding/prod0/specimen_791855403/isi_experiment_802369734/',
  'workflow_state': 'passed',
  'name': '429956_20190102',
  'id': 802369734},
 'name': '20190327_429956_Ophys1',
 'workflow_state': 'uploaded',
 'specimen_id': 791855403,
 'date_of_acquisition': datetime.datetime(2019, 3, 27, 8, 37, 11),
 'depth': None,
 'project_code': 'VisualBehaviorMultiscope',
 'area': None,
 'storage_directory': '/allen/programs/braintv/production/neuralcoding/prod0/specimen_791855403/ophys_session_842023261/',
 'ophys_parent_session': {'storage_directory': '/allen/programs/braintv/production/neuralcoding/prod0/specimen_791855403/ophys_session_841682738/',
  'workflow_state': 'uploaded',
  'name': '20190326_429956_ophys1',
  'id': 841682738},
 'operator': 'sams',
 'rig': 'MESO.1',
 'ophys_experiments': [{'storage_directory': '/allen/programs/braintv/production